#### This is the second iteration of BERT tuning to see if we can imporove it a bit further, if we cannot we will settle for the best model we get

In [1]:
# 06b_bert_finetune_weighted.ipynb
# Cell 1 – set up project root, src path, and config

from pathlib import Path
import sys

# Detect project root (assuming this notebook lives in notebooks/)
cwd = Path.cwd().resolve()
if cwd.name == "notebooks":
    PROJECT_ROOT = cwd.parent
else:
    PROJECT_ROOT = cwd

print("PROJECT_ROOT:", PROJECT_ROOT)

# Make sure src/ is on the Python path
SRC_DIR = PROJECT_ROOT / "src"
if not SRC_DIR.is_dir():
    raise FileNotFoundError(f"'src' folder not found at {SRC_DIR}")

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import shared config (paths)
from config import PROCESSED_DATA_DIR

print("PROCESSED_DATA_DIR:", PROCESSED_DATA_DIR)

PROJECT_ROOT: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation
PROCESSED_DATA_DIR: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/data/processed


In [2]:
# Cell 2 – load processed products and inspect splits

import pandas as pd

products_path = PROCESSED_DATA_DIR / "products_with_splits.parquet"
df = pd.read_parquet(products_path)

print("Full dataset shape:", df.shape)
print("\nSplit counts:")
print(df["split"].value_counts())

print("\nColumns:")
print(df.columns.tolist())

# Quick peek at the key columns
df[["product_text_raw", "product_text_norm", "label", "split"]].head()

Full dataset shape: (19767, 10)

Split counts:
split
train    13961
val       3262
test      2544
Name: count, dtype: int64

Columns:
['event_id', 'event_type', 'start_time_local', 'remove_amazon', 'month', 'product_text_raw', 'product_text_norm', 'label_raw', 'label', 'split']


,product_text_raw,product_text_norm,label,split
0,001B 3000A Car Jump Starter Battery Pack (up t...,001b 3000a car jump starter battery pack up to...,0,train
1,"012 Jump Starter Battery Pack, 4000A Peak Car ...",012 jump starter battery pack 4000a peak car b...,0,train
2,1/2 Ct Diamond Stud Earrings 14k Yellow Gold F...,1 2 ct diamond stud earrings 14k yellow gold f...,1,test
3,1-2 Pairs 925 Sterling Silver Mens Earrings Cu...,1 2 pairs 925 sterling silver mens earrings cu...,1,train
4,"1/2"""" x 18"""" Zirconia Sanding Belts for Belt S...",1 2 x 18 zirconia sanding belts for belt sande...,0,train


In [3]:
# Cell 3 – experiment config and tokenizer setup

from transformers import AutoTokenizer
import torch

# New experiment name so we DO NOT overwrite the baseline model
EXPERIMENT_NAME = "bert_fashion_weighted"
OUTPUT_DIR = PROJECT_ROOT / "models" / EXPERIMENT_NAME

print("Experiment name:", EXPERIMENT_NAME)
print("Model output dir:", OUTPUT_DIR)

# BERT backbone and sequence length (same as baseline)
MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 64

# Class weights: [weight_for_label_0, weight_for_label_1]
# We care more about non-fashion (0), so we give it a higher weight.
CLASS_WEIGHTS = torch.tensor([2.0, 1.0])

print("MODEL_NAME:", MODEL_NAME)
print("MAX_LENGTH:", MAX_LENGTH)
print("CLASS_WEIGHTS:", CLASS_WEIGHTS.tolist())

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Quick sanity check on tokenization length
sample_text = df.loc[0, "product_text_norm"]
enc = tokenizer(
    sample_text,
    padding="max_length",
    truncation=True,
    max_length=MAX_LENGTH,
    return_tensors="pt",
)

print("\nSample text:", sample_text[:120], "...")
print("input_ids length:", enc["input_ids"].shape[1])

/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Experiment name: bert_fashion_weighted
Model output dir: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/models/bert_fashion_weighted
MODEL_NAME: bert-base-uncased
MAX_LENGTH: 64
CLASS_WEIGHTS: [2.0, 1.0]

Sample text: 001b 3000a car jump starter battery pack up to 9 0l gas and 7 0l diesel engine 12v car battery charger jump box with usb ...
input_ids length: 64


In [4]:
# Cell 4 – build HF datasets and tokenize (same splits as baseline)

from datasets import Dataset, DatasetDict

# 1. Split the dataframe into train / val / test
train_df = df[df["split"] == "train"].reset_index(drop=True)
val_df   = df[df["split"] == "val"].reset_index(drop=True)
test_df  = df[df["split"] == "test"].reset_index(drop=True)

print("Train shape:", train_df.shape)
print("Val shape  :", val_df.shape)
print("Test shape :", test_df.shape)

# 2. Create Hugging Face Dataset objects
def df_to_hf(ds_df):
    return Dataset.from_pandas(
        ds_df[["product_text_norm", "label"]].rename(
            columns={"product_text_norm": "text"}
        )
    )

train_hf = df_to_hf(train_df)
val_hf   = df_to_hf(val_df)
test_hf  = df_to_hf(test_df)

print("\nRaw HF datasets:")
print("  train:", train_hf)
print("  val  :", val_hf)
print("  test :", test_hf)

# 3. Tokenization function
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
    )

# 4. Apply tokenization
train_tok = train_hf.map(tokenize_batch, batched=True)
val_tok   = val_hf.map(tokenize_batch, batched=True)
test_tok  = test_hf.map(tokenize_batch, batched=True)

# Set format for PyTorch
cols = ["input_ids", "attention_mask", "token_type_ids", "label"]
train_tok.set_format(type="torch", columns=cols)
val_tok.set_format(type="torch", columns=cols)
test_tok.set_format(type="torch", columns=cols)

print("\nTokenized HF datasets:")
print("  train_tok:", train_tok)
print("  val_tok  :", val_tok)
print("  test_tok :", test_tok)

# Quick sanity check on one example
example = train_tok[0]
print("\nExample keys:", example.keys())
print("input_ids length:", len(example["input_ids"]))
print("label:", example["label"].item())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Train shape: (13961, 10)
Val shape  : (3262, 10)
Test shape : (2544, 10)

Raw HF datasets:
  train: Dataset({
    features: ['text', 'label'],
    num_rows: 13961
})
  val  : Dataset({
    features: ['text', 'label'],
    num_rows: 3262
})
  test : Dataset({
    features: ['text', 'label'],
    num_rows: 2544
})


Map: 100%|██████████| 2544/2544 [00:00<00:00, 38513.33 examples/s]


Tokenized HF datasets:
  train_tok: Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13961
})
  val_tok  : Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3262
})
  test_tok : Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2544
})

Example keys: dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask'])
input_ids length: 64
label: 0


In [5]:
# Cell 5 – define weighted BERT model, Trainer, and training args

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch.nn as nn
import numpy as np

# Load base BERT classification model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)

# Move class weights to same device later (in compute_loss)
CLASS_WEIGHTS = CLASS_WEIGHTS.to(torch.float32)

# Metrics function (same as baseline, simple accuracy + F1)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
        "f1": f1_score(labels, preds),
    }

# Custom Trainer to use weighted cross-entropy
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Standard forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Weighted cross-entropy: weight[0] for non-fashion, weight[1] for fashion
        loss_fct = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Training arguments – note output_dir is the NEW experiment folder
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=3,              # same as baseline
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=50,
    save_total_limit=2,
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)

print("Trainer ready. Output dir will be:", training_args.output_dir)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Trainer ready. Output dir will be: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/models/bert_fashion_weighted


#### 	•	In this run we trained a second BERT model (bert_fashion_weighted) with the same architecture, tokenizer, data splits, max length (64), learning rate schedule, batch size, and number of epochs as the original bert_fashion model.
####	•	The only change was the loss weighting: we set class_weights = [2.0, 1.0], giving class 0 (non-fashion, the minority class) twice the weight of class 1 (fashion). This makes the model “care more” about mistakes on non-fashion items and is aimed at reducing cases where non-fashion products are predicted as fashion.
####	•	All evaluation and QA will reuse the same validation/test splits and the same 100-row unlabeled QA sample, so we can directly compare the original BERT model, the weighted BERT model, and the logistic baseline on exactly the same data.

In [8]:
# Cell 6 – train weighted BERT and do a quick sanity eval

# Train the model (this will create models/bert_fashion_weighted/)
train_result = trainer.train()

# Explicitly save model + tokenizer to the new experiment folder
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("\nTraining finished.")
print("Best model saved in:", OUTPUT_DIR)

# Quick sanity check on val and test (no plots, just numbers)
print("\n=== Evaluation on validation split ===")
val_metrics = trainer.evaluate(eval_dataset=val_tok)
print(val_metrics)

print("\n=== Evaluation on test split ===")
test_metrics = trainer.evaluate(eval_dataset=test_tok)
print(test_metrics)

  0%|          | 0/2619 [00:00<?, ?it/s]/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
  2%|▏         | 51/2619 [00:14<08:21,  5.13it/s] 

{'loss': 0.3672, 'grad_norm': 8.153903007507324, 'learning_rate': 1.9618174875906837e-05, 'epoch': 0.06}


  4%|▍         | 101/2619 [00:23<08:07,  5.17it/s]

{'loss': 0.2663, 'grad_norm': 6.130841255187988, 'learning_rate': 1.923634975181367e-05, 'epoch': 0.11}


  6%|▌         | 151/2619 [00:33<07:53,  5.21it/s]

...

{'train_runtime': 542.9006, 'train_samples_per_second': 77.147, 'train_steps_per_second': 4.824, 'train_loss': 0.09088216179789091, 'epoch': 3.0}

Training finished.
Best model saved in: /Users/ramana/Documents/Homework/1st class ML opt/Project 1/Product-Classifcation/models/bert_fashion_weighted

=== Evaluation on validation split ===


/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 102/102 [00:09<00:00, 11.16it/s]
/Users/ramana/anaconda3/envs/fashion-bert/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.10478895157575607, 'eval_accuracy': 0.9834457388105456, 'eval_precision': 0.992790194664744, 'eval_recall': 0.9878048780487805, 'eval_f1': 0.9902912621359223, 'eval_runtime': 9.2634, 'eval_samples_per_second': 352.137, 'eval_steps_per_second': 11.011, 'epoch': 3.0}

=== Evaluation on test split ===


100%|██████████| 80/80 [00:07<00:00, 11.23it/s]

{'eval_loss': 0.20732198655605316, 'eval_accuracy': 0.9713050314465409, 'eval_precision': 0.9810066476733144, 'eval_recall': 0.9842782277274893, 'eval_f1': 0.9826397146254459, 'eval_runtime': 7.209, 'eval_samples_per_second': 352.892, 'eval_steps_per_second': 11.097, 'epoch': 3.0}
